In [1]:
# =============================
# Data Extraction Notebook
# Project: Energy Prices in Illinois
# Milestone 1
# =============================

In [2]:
# Import required libraries
import pandas as pd
import requests
from io import StringIO
import pickle
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# -----------------------------
# 1. Define Save Paths
# -----------------------------
BASE_PATH = "/content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois"
RAW_PATH = "/content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois/Data/raw"
PROC_PATH = "/content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois/Data/processed"
os.makedirs(RAW_PATH, exist_ok=True)
os.makedirs(PROC_PATH, exist_ok=True)

In [5]:
# -----------------------------
# 2. Define Data Sources
# -----------------------------
# Actual Energy Price (label)
ACTUAL_PRICE_URL = "http://www.energyonline.com/Data/GenericData.aspx?DataId=8&MISO___Actual_Energy_Price"

# Day Ahead Energy Price (feature)
DAY_AHEAD_PRICE_URL = "http://www.energyonline.com/Data/GenericData.aspx?DataId=8&MISO___Day_Ahead_Energy_Price"

In [6]:
# -----------------------------
# 3. Helper Function
# -----------------------------
def fetch_energy_table(url, name="data"):
    print(f"Fetching {name} from {url} ...")
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception(f"Failed to fetch {name}, status {r.status_code}")

    # Save raw HTML
    raw_file = os.path.join(RAW_PATH, f"{name.lower().replace(' ', '_')}.html")
    with open(raw_file, "w") as f:
        f.write(r.text)
    print(f"💾 Saved raw HTML -> {raw_file}")

    # Parse tables
    dfs = pd.read_html(StringIO(r.text))
    print(f"Found {len(dfs)} tables for {name}")

    # Pick first non-empty
    df = None
    for candidate in dfs:
        if candidate.shape[0] > 0:
            df = candidate
            break
    if df is None:
        raise ValueError(f"No valid tables found for {name}")

    # Clean column names (force strings)
    df.columns = [str(c).strip().lower().replace(" ", "_") for c in df.columns]

    # Normalize: first col = date, second col = value
    if df.shape[1] >= 2:
        df.rename(
            columns={df.columns[0]: "date", df.columns[1]: f"{name.lower().replace(' ', '_')}"},
            inplace=True
        )
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # Save CSV
    csv_file = os.path.join(RAW_PATH, f"{name.lower().replace(' ', '_')}.csv")
    df.to_csv(csv_file, index=False)
    print(f"💾 Saved raw CSV -> {csv_file}")

    print(df.head())  # 👀 Quick preview so we know columns
    return df

In [7]:
# -----------------------------
# 4. Fetch Datasets
# -----------------------------
df_actual = fetch_energy_table(ACTUAL_PRICE_URL, "Actual Energy Price")
df_day_ahead = fetch_energy_table(DAY_AHEAD_PRICE_URL, "Day Ahead Energy Price")

Fetching Actual Energy Price from http://www.energyonline.com/Data/GenericData.aspx?DataId=8&MISO___Actual_Energy_Price ...
💾 Saved raw HTML -> /content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois/Data/raw/actual_energy_price.html
Found 17 tables for Actual Energy Price
💾 Saved raw CSV -> /content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois/Data/raw/actual_energy_price.csv
  date  actual_energy_price          2   3   4
0  NaT                  NaN        NaN NaN NaN
1  NaT                  NaN        NaN NaN NaN
2  NaT                  NaN        NaN NaN NaN
3  NaT                  NaN  End Date: NaN NaN
4  NaT                  NaN        NaN NaN NaN
Fetching Day Ahead Energy Price from http://www.energyonline.com/Data/GenericData.aspx?DataId=8&MISO___Day_Ahead_Energy_Price ...


/tmp/ipython-input-2391821720.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


💾 Saved raw HTML -> /content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois/Data/raw/day_ahead_energy_price.html
Found 17 tables for Day Ahead Energy Price
💾 Saved raw CSV -> /content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois/Data/raw/day_ahead_energy_price.csv
  date  day_ahead_energy_price          2   3   4
0  NaT                     NaN        NaN NaN NaN
1  NaT                     NaN        NaN NaN NaN
2  NaT                     NaN        NaN NaN NaN
3  NaT                     NaN  End Date: NaN NaN
4  NaT                     NaN        NaN NaN NaN


/tmp/ipython-input-2391821720.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


In [8]:
# -----------------------------
# 5. Merge & Add Features
# -----------------------------
if not df_actual.empty and not df_day_ahead.empty:
    df = pd.merge(
        df_actual, df_day_ahead,
        on="date", how="inner", suffixes=("_actual", "_dayahead")
    )

    # Calendar features
    df["day_of_week"] = df["date"].dt.day_name()
    df["hour_of_day"] = df["date"].dt.hour
    df["month"] = df["date"].dt.month

    print("✅ Merged dataset:", df.shape)
else:
    print("❌ One of the datasets is empty.")
    df = pd.DataFrame()

✅ Merged dataset: (64, 12)


In [9]:
# -----------------------------
# 6. Debug & Working Datasets
# -----------------------------
df_debug = df.head(500)      # small subset for quick testing
df_working = df.copy()       # full dataset

In [10]:
# -----------------------------
# 7. Save Processed Data
# -----------------------------
debug_pickle = os.path.join(PROC_PATH, "energy_debug.pkl")
working_pickle = os.path.join(PROC_PATH, "energy_working.pkl")

df_debug.to_pickle(debug_pickle)
df_working.to_pickle(working_pickle)

print(f"💾 Saved processed debug -> {debug_pickle}")
print(f"💾 Saved processed working -> {working_pickle}")

💾 Saved processed debug -> /content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois/Data/processed/energy_debug.pkl
💾 Saved processed working -> /content/drive/MyDrive/ IE434-DeepDive-EnergyPricesInIllinois/Data/processed/energy_working.pkl
